## Anotações e Input/output de sequências



Tópicos:

* Objeto SeqRecord
* Módulo SeqIO.parse
* Módulo SeqIO.write

####Recapitulando 

Na aula anterior aprendemos a criar e manipular o objeto sequência (Seq), bem como a vantagens de ter um objeto ao invés de usar uma string

In [4]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 7.2 MB/s 


In [ ]:
from Bio.Seq import Seq

#criando o objeto Seq
my_seq = Seq("AGTACACTGGT")
print(my_seq)

#usando médodos do objeto
print(my_seq.complement())
print(my_seq.reverse_complement())



No entanto, na maioria das vezes não queremos criar essas sequências e sim manipular sequências a partir de algum arquivo externo. 
Nessa aula iremos aprender a importar arquivos para o biopython, visualizar anotações e escrever arquivos nos diferentes formatos biológicos.

# Importando arquivos de sequências

Parte da vida de um bioinformata é lidar com os 3x10^26 formatos de arquivos diferentes, cada qual com seu tipo de dado biológico, estrutura de dados e formatação únicos. Para facilitar sua vida e lhe poupar frustração de ter que escrever "parsers" para todos esses arquivos, o biopython já vem com suporte a maioria dos formatos de dados biológicos (Fasta, [Genbank](http://www.genebio.ufba.br/genbank/))

In [ ]:
## nota: o colab reseta todas as vezes que desliga então vamos puxar a página 
## da web como arquivo. Não é necessário se estiver trabalhando com arquivos locais

from urllib.request import urlretrieve

# urlretrieve("url do arquivo", "nome do arquivo")
urlretrieve("https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna", "fasta.fna")
urlretrieve("https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.gbk", "orchid.gbk")


Iremos discutir o pacote **Bio.SeqIO**. O objetivo desse pacote é fornecer uma interface simples para trabalhar com diversos formatos de arquivo de sequência de maneira uniforme. Mas **ATENÇÃO**: esse pacote lida apenas com sequências que são **objetos SeqRecord** (o qual contém objetos Seq com anotações e falaremos em breve).

In [5]:
from Bio import SeqIO

A principal função desse módulo, **Bio.SeqIO.parse()** é usada para ler dados de sequências como **objetos SeqRecord**. Esta função espera **dois argumentos**:

1. O primeiro argumento é um **handle (identificador)** para ler os dados ou um nome de arquivo. Um handle é normalmente um arquivo aberto para leitura, mas pode ser a saída de um programa de linha de comando ou dados baixados da rede.

2. O segundo argumento é uma string com letras minúsculas especificando o **formato** da sequência - consulte http://biopython.org/wiki/SeqIO para obter uma lista completa dos formatos suportados.

A função Bio.SeqIO.parse() retorna um **iterador** que fornece objetos SeqRecord. Iteradores são normalmente usado em um " for loop" como no exemplo a seguir:


In [ ]:
from Bio import SeqIO

for seq_record in SeqIO.parse("fasta.fna", "fasta"):
  print(seq_record.id)
  print(repr(seq_record.seq))
  print(len(seq_record))



In [ ]:
# recapitulando da aula de ontem
'''
class Transgenico:
  def __init__(self, organismo, dna_organismo=None):
    self.organismo = organismo
    self.dna_organismo = dna_organismo
    
milho = Transgenico("milho")
milho.dna_organismo = "ACATCTGATCAGCTACGATCGATCAGCTAGCAT"
print(milho)
'''

Note que você *precisa* especificar o formato do arquivo.

O objeto retornado por Bio.SeqIO é na verdade um iterador que retorna objetos SeqRecord. Você consegue ver cada registro por vez, mas apenas uma vez. O ponto positivo é que um iterador pode economizar memória ao lidar com arquivos grandes.

Em vez de usar um loop for, também pode usar a função nativa **next()** em um iterador para percorrer as entradas.

In [ ]:
record_iterator = SeqIO.parse("orchid.gbk", "gb")

# usar next() para ir iterando sequência por sequência
first_record = next(record_iterator)
print(first_record.id)
print(first_record.description)

In [ ]:
print(next(record_iterator).id)
print(first_record.id)

Observe que se você tentar usar **next()** e não houver mais resultados, você obterá a exceção "StopIteration".

### **treino**
Conte quantas sequências do arquivo orchid.gbk tem mais de 700 aminoácidos.

In [ ]:
### escreva seu código aqui

#### Solução

In [ ]:
## solução exemplo
from Bio import SeqIO
arquivo = "orchid.gbk"
total = 0
count = 0
for record in SeqIO.parse(arquivo, "genbank"):
    total = total + 1
    if len(record.seq) > 700:
      count += 1
print("há " + str(total) + " sequencias no arquivo " + arquivo + " das quais "+ str(count) + " tem mais de 700 aminoácidos")

há 94 sequencias no arquivo orchid.gbk das quais 74 tem mais de 700 aminoácidos


# Objeto SeqRecord

Na aula passada introduzimos a classe Seq. Acontece que a classe Seq faz parte de outra classe, a  **Sequence Record** ou
**SeqRecord**, definida no módulo **Bio.SeqRecord**. Essa classe permite utilizar recursos de níveis mais altos como identificadores e "features" associados às sequencias, bem como facilita a conversão de formatos.

A classe SeqRecord tem os atributos:

* seq – A sequencia, tipicamente um objeto **Seq**.
* id – O ID primário usado para identificar a sequencia – **string**

* name – Análogo ao LOCUS no arquivo do GenBank, pode ser um nome comum ou um clone do ID - **string**
* description – Uma descrição 'entendível' por humanos - **string**.
* letter annotations – **Dicionário** onde as chaves são o nome da informação e o valor é uma sequencia em python (lista, tupla ou string) com o mesmo tamanho da sequência. Normalmente usado para scores de qualidade de alinhamento ou informações de estrutura secundária (arquivos de alinhamento Stockholm/PFAM, fora do escopo desse minicurso).

* annotations – Dicionário de informações adicionais sobre a sequência, permitindo adição de informações mais "desestruturadas. - **dict**
* features – Uma lista de **objetos SeqFeature** com dados mais estruturados
(posição de genes no genoma, domínios em uma sequencia protéica, etc).
* dbxrefs - referencias cruzadas em databases - **lista de strings**

Note que nem todos os atributos precisam ter algo

In [ ]:
for seq_record in SeqIO.parse("orchid.gbk", "gb"):
  print(seq_record.id)
  print(seq_record.name)
  print(seq_record.description)
  print(seq_record.letter_annotations)
  print(seq_record.annotations)
  print(seq_record.features)
  print(seq_record.dbxrefs)
  

### Criando um objeto seqRecord

para criar um objeto SeqRecord você só precisa ter inicialmente um objeto Seq

In [ ]:
from Bio.Seq import Seq
seq_simples = Seq("GATC")

from Bio.SeqRecord import SeqRecord
seq_anotada = SeqRecord(seq_simples)

depois de criado, é possível popular os outros atributos com informações

In [ ]:
seq_anotada.id = "BP1234"
seq_anotada.name ="minhaSeqAnotada"
seq_anotada.description = "minha linda descrição"
print(seq_anotada.format("fasta"))

Para adicionar itens no anottations (e no letter) é só lembrar que ambos são dicionários normais. 

In [ ]:
#seq_anotada.letter_annotations["phred_quality"] = [40, 40, 38, 30]
seq_anotada.annotations["organism"] = "unicorn"
seq_anotada.annotations["source"] = "minha imaginação"
print(repr(seq_anotada))

print(seq_anotada.annotations)

SeqRecord(seq=Seq('GATC'), id='BP1234', name='minhaSeqAnotada', description='minha linda descrição', dbxrefs=[])
{'organism': 'unicorn', 'source': 'minha imaginação'}


In [ ]:
#print(seq_anotada.format("genbank"))

Repare que apesar de ser possível colocar em tese qualquer coisa como anotação, não será possível converter para outros formatos que não compartilhem a mesma estrutura de chaves de dicionário.

Vamos importar mais um arquivo para continuar essa sessão. Dessa vez usaremos um exemplo com valores em todos os atributos. 

In [ ]:
from urllib.request import urlretrieve
urlretrieve("https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.gb", "single_seq.gbk")

Sabemos que esse arquivo tem uma única sequencia, poderíamos cuntinuar usando o for loop e o **SeqIO.parse()** mas o biopython oferece uma opção em casos especiais com apenas uma sequencia: o **SeqIO.read()**.

In [ ]:
from Bio import SeqIO
record = SeqIO.read("single_seq.gbk", "gb")
print(record.id + " length " + str(len(record)))

*Eu sei que você agora quer tentar usar o read com os outros arquivos, então vá em frente e tente!*

In [ ]:
print(record.name)
print(record.annotations)
print(record.annotations['organism'])

#### Treino 2: 
Sem abrir o arquivo no navegador, acesse o arquivo "sequence.gb" e obtenha informações do organismo

In [ ]:
#Importando o arquivo como 
urlretrieve("https://raw.githubusercontent.com/biopyladies/Minicurso-biopython-wbds/main/notebooks/arquivos/sequence.gb", "sequence.gbk")
#Escreva seu código abaixo


# Classe SeqFeature

É com essa classe que se cria o atributo feature do SeqRecord. A classe SeqFeature tenta encapsular o máximo possível de informações sobre a sequência. O design é fortemente baseado nas tabelas de recursos do GenBank / EMBL.

A ideia principal sobre cada objeto SeqFeature é descrever uma região em uma sequência específica. Essa região é descrita com um objeto de localização, normalmente um intervalo entre duas posições (mRna, gene, exons, introns, etc).

Atributos da classe SeqFeature:

1. **type**: descreve o tipo de feature, ex: gene, mRNA ...

2. **locations**: para descrever as regiões da sequência, usamos o atributo locations e outros métodos que veremos mais para frente.
3. **qualifiers**: é um dicionário Python de informações adicionais. ex: uma chave poderia ser ["evidência"] = "não experimental" para indicar que o gene foi encontrado de maneira computacional.
4. **sub_features**: IGNOREM ESSE ATRIBUTO. Se tornou obsoleto com a criação do objeto CompoundLocation (veremos a seguir).

In [6]:
from Bio.SeqFeature import SeqFeature

**Objeto FeatureLocation**

Em genes de procarioto, a maioria das localizações do SeqFeature são extremamente simples - você só precisa de coordenadas de início e fim e uma fita. Isso é essencialmente tudo o que o objeto FeatureLocation básico faz.

Em eucariotos, há altas chances de termos que lidar com locais compostos (óperons). Além disso, as próprias posições podem ser confusas (inexatas). Para isso, foi criado o **Objeto CompoundLocation**, que junta regiões de genes em locais diferentes.

Para analisar genes com posições não tão certas, temos o conceito de **posições fuzzy**, mas como é muito específico, não trouxe aqui. Para saber mais: [4.3.2.3 do Manual Python](https://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Alocations/) 



Observe que CDS do GenBank ou arquivos EMBL é a união dos exons - eles não cobrem nenhum íntron.

In [ ]:
record = SeqIO.read("single_seq.gbk", "gb")
for feature in record.features:
  print(feature)

Print só os tipos (atributo type)

In [ ]:
for feature in record.features:
  print(feature.type)

Print só os dicionários com informações dos tipos (atributo qualifiers)

In [ ]:
for feature in record.features:
  print(feature.qualifiers)

Procurando uma informação no features

In [ ]:
#pesquisando se  é a informação de algum tipo de feature, se sim, qual o tipo e a localização
for feature in record.features:
  if ["hypothetical protein"] in feature.qualifiers.values():
    print(feature.type, feature.location)

## Fatiando objetos SeqRecord (Slicing SeqRecord)

Selecionando uma parte do objeto SeqRecord e o transformando em outro objeto SeqRecord

In [ ]:
other_record = record.features[4].location
print(other_record)

In [ ]:
other_record = record[86:959]
type(other_record)
print(other_record)

Algumas informações sobre o SeqRecord original são mantidas para o novo SeqRecord, porém é necessário fazer algumas atualizações para informações que não fazem sentido.

In [ ]:
print(other_record.id) #mesmo id
print(other_record.seq)
print(other_record.description) # não é mais um plasmidio completo
print(other_record.annotations)
print(other_record.features)

In [ ]:
other_record.description = "Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1. Misc feature"
print(other_record)

## Extrair Location

Um SeqFeature ou objeto de "location" não contém a sequencia direta, mas direciona para como conseguir extrair a sequencia da sequencia original. Para esse exemplo imagine um gene com uma "location" 5:18 na fita reversa:

In [ ]:
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
seq = Seq("ACCGAGACGGCAAAGGCTAGCATAGGTATGAGACTTCCTTCCTGCCAGTGCTGAGGAACTGGGAGCCTAC")
feature = SeqFeature(FeatureLocation(5, 18), type="gene", strand=-1)
##### Sim, pulamos antes como criar um SeqFeature para simplificar
##### sem bancada não se encontra location!

Você poderia pegar manualmente, fazer o reverso complemento e o slicing e funcionaria, porém o biopython já te cobre com o método **extract**

In [ ]:
#manualmente
feature_seq = seq[feature.location.start:feature.location.end].reverse_complement()
print(feature_seq)


feature_seq = feature.extract(seq)
print(feature_seq)


AGCCTTTGCCGTC
AGCCTTTGCCGTC


## Comparando SeqRecords

Em resumo, não é possível. *Compare os atributos!*

In [ ]:
record1 = SeqRecord(Seq("ACGTTTTTCAGCTAGCAAAAAATCCCCAAATC"), id="A12")
record2 = SeqRecord(Seq("ACGTTTTTCAGCACTAGCTAGATTAGCATC"), id="A25")

#record1 == record2 isso vai dar errado

record1.seq == record2.seq

### Treino 3:
Usando o arquivo "sequence.gbk", encontre a location da "plasmid recombination protein" e extraia a sequencia

In [25]:
# seu código aqui

###### Solução:

In [ ]:
record2 = SeqIO.read("sequence.gbk", "gb")
for feature in record2.features:
  if ["plasmid recombination protein"] in feature.qualifiers.values():
    print(feature.location)
    print(feature.extract(record2.seq))
    

# Salvando SeqRecords em um dicionário

Até aqui usamos variáveis e iteradores para acessar os dados, mas ao se trabalhar com uma longa coleção de sequencias, você pode querer ter alguma forma de manter esses dados de forma organizada na memória


 ainda três funções relacionadas no módulo Bio.SeqIO que permitem um colocar em um **dicionário** um arquivo de várias sequências. **Há um trade off aqui entre flexibilidade e uso de memória**. 

• **SeqIO.to_dict()** é a opção **mais flexível**, mas também a que **mais exige memória**. Esta é basicamente uma função auxiliar para construir um dicionário Python normal com cada entrada mantida como um objeto SeqRecord na memória, permitindo que você modifique os registros. (por padrão a chave será o ID d cada registro)

In [ ]:
orchid_dict = SeqIO.to_dict(SeqIO.parse("orchid.gbk", "gb"))


• **SeqIO.index()** é um **meio termo útil**, agindo **"como um dicionário" somente leitura** e analisando sequências em objetos SeqRecord sob demanda.


In [ ]:
orchid_dict = SeqIO.index("orchid.gbk", "gb")


• **SeqIO.index_db()** também atua **"como um dicionário" somente leitura**, mas armazena os identificadores e a localização de arquivo (file offsets) em um arquivo no disco (como um banco de dados SQLite3), o que significa que tem **requisitos de memória muito baixos**, mas é **um pouco mais lento**.

In [ ]:
# vamos baixar primeiro um arquivo grande para ele indexar
!curl -O ftp://ftp.ncbi.nih.gov/genbank/gbvrl1.seq.gz
!curl -O ftp://ftp.ncbi.nih.gov/genbank/gbvrl2.seq.gz
!curl -O ftp://ftp.ncbi.nih.gov/genbank/gbvrl3.seq.gz
!curl -O ftp://ftp.ncbi.nih.gov/genbank/gbvrl4.seq.gz

# descompactar
!gunzip gbvrl*.seq.gz
import glob 
files = glob.glob("gbvrl*.seq")

gb_vrl = SeqIO.index_db("gbvrl.idx", files, "genbank")
print(f"{len(gb_vrl)} sequências indexadas")



N exemplo, considere as versões de arquivo simples do GenBank do site FTP do NCBI, ftp://ftp.ncbi.nih.gov/genbank/, que são arquivos GenBank compactados com gzip. Na versão 210 do GenBank, há 38 arquivos constituindo as sequências virais, gbvrl1.seq,. . . , gbvrl38.seq, ocupando cerca de 8 GB no disco depois de descompactado e contendo no total quase dois milhões de registros....

A indexação dos primeiros quatro arquivos demora cerca de um minuto. 
No entanto, uma vez feito, repetir isso recarregará o arquivo gbvrl.idx em uma fração de segundo. Você pode usar o índice como um dicionário Python somente leitura, sem precisar se preocupar com o arquivo de origem da sequência, por exemplo:


In [ ]:
print(gb_vrl["AB811634.1"].description)

Equine encephalosis virus NS3 gene, complete cds, isolate: Kimron1


No mais, todos os 3 médotos de indexação darão o mesmo resultado e podem ser usados da mesma forma após criados (são dicionários normais!)

# Escrevendo arquivos

Para escrever arquivos de sequencias usamos a função **Bio.SeqIO.write()**, que tem como argumentos um iterador (ou lista) SeqRecord, um handle para o output (ou nome do arquivo), e o formato:

In [ ]:
SeqIO.write(seq_anotada, "exemplo.fasta", "fasta")

1

Você também pode usar a função Bio.SeqIO.convert() para converter entre 2 quaisquer formatos. (*atenção à perda de informação*)

O módulo convert exige 4 argumentos 
* caminho do arquivo original
* formato do arquivo original
* nome do novo arquivo
* formato do novo arquivo

In [ ]:
SeqIO.convert("orchid.gbk", "genbank", "orchid.fasta", "fasta")

###Desafio final
acesse o arquivo orchid.gbk e salve em um arquivo fasta todos os registros do gênero *Cypripedium*.

In [ ]:
# escreva aqui seu código

#### Solução

In [ ]:
from Bio import SeqIO

Seqs = []
for seq_record in SeqIO.parse("orchid.gbk", "gb"):
  if "Cypripedium" in seq_record.annotations['taxonomy']:
    Seqs.append(seq_record)

SeqIO.write(Seqs, "Cypripedium.fasta", "fasta")




# Cookbook

In [ ]:
# iterador - FASTA
for record in SeqIO.parse("fasta.fna", "fasta"):
  print(f"ID: {record.id}    Tamanho: {len(record.seq)} nt")

In [ ]:
# iterador - GENBANK
for record in SeqIO.parse("orchid.gbk", "gb"):
  print(f"ID: {record.id}    Tamanho: {len(record.seq)} nt")

In [ ]:
# Salvando a informação do iterador em uma lista
ids = [record.id for record in SeqIO.parse("orchid.gbk", "gb")]
print(ids)
# indexando em um dicionário
orchid_dict = SeqIO.index("orchid.gbk", "gb")
# recuperando informação
seq_record = orchid_dict["Z78475.1"]
print(seq_record.description)
print(seq_record.seq)

In [ ]:
# Convertendo arquivos
count = SeqIO.convert("orchid.gbk", "genbank", "orchid.fasta", "fasta")
print("Converted %i records" % count)

# Para ir além

Esse colab não esgotou o assunto, ainda há muitas funções e truques legais que o biopython pode fazer

* usar um parser de baixo nível para rodar arquivos de sequenciamento e high-throughput
* baixar os arquivos diretamente dos bancos de dados GenBank, SwissProt 
* Indexar e trabalhar com arquivos compactados muuuito grandes

Consulte aqui a documentação. Nesta aula apresentamos parte dos conteúdos do capítulo 4 e 5 do livro biopython, na próxima aula iremos explorar os bancos de dados  online.

# **Feedback Minicurso**

[DIA 2](https://docs.google.com/forms/d/e/1FAIpQLSeH6LofkWSbzcrwo8Q51N5FsuX3oOneBwAlb8PvPXFe_Fyn_Q/viewform?usp=sf_link)